### Import Libs

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

### Check data (JSON file)

In [2]:
df = pd.read_json('Challenge2_data.json')
df

,Apparent Magitude (m),Absolute Magnitude (M),Redshift (z)
0,14.541391,-16.339065,0.003676
1,16.756129,-18.177730,0.022604
2,15.829605,-16.696145,0.008038
3,16.294575,-19.512265,0.036352
4,13.829094,-19.665756,0.010395
5,14.182569,-21.249230,0.033587
6,16.039077,-17.778063,0.014448
7,17.138868,-17.656339,0.017838
8,15.998251,-19.397655,0.030485
9,17.910487,-16.931928,0.025338


In [3]:
df.columns

Index(['Apparent Magitude (m)', 'Absolute Magnitude (M)', 'Redshift (z)'], dtype='object')

### Rename columns

In [4]:
df.rename(columns = {'Apparent Magitude (m)':'m', 'Absolute Magnitude (M)':'M', 'Redshift (z)':'z'}, inplace = True)
df

,m,M,z
0,14.541391,-16.339065,0.003676
1,16.756129,-18.177730,0.022604
2,15.829605,-16.696145,0.008038
3,16.294575,-19.512265,0.036352
4,13.829094,-19.665756,0.010395
5,14.182569,-21.249230,0.033587
6,16.039077,-17.778063,0.014448
7,17.138868,-17.656339,0.017838
8,15.998251,-19.397655,0.030485
9,17.910487,-16.931928,0.025338


### Raw data to numpy array

In [5]:
m = df['m'].to_numpy()
M = df['M'].to_numpy()
z = df['z'].to_numpy()
m, M, z

(array([14.54139124, 16.75612879, 15.82960489, 16.29457481, 13.82909389,
        14.18256872, 16.03907735, 17.13886801, 15.9982508 , 17.91048687,
        15.07466972, 15.16403959, 12.48463919, 15.6016586 , 13.75361766,
        14.50453086, 14.14702991, 16.04484713, 15.19214105]),
 array([-16.33906506, -18.17772989, -16.696145  , -19.5122652 ,
        -19.66575613, -21.24923043, -17.77806262, -17.65633895,
        -19.39765543, -16.93192787, -20.91861571, -18.86686028,
        -23.32220082, -18.32499057, -21.31056846, -18.8092583 ,
        -18.51036467, -20.29101151, -16.31300893]),
 array([0.00367597, 0.02260424, 0.00803785, 0.03635172, 0.01039452,
        0.03358662, 0.01444763, 0.017838  , 0.03048528, 0.02533826,
        0.0301413 , 0.017838  , 0.03255174, 0.01106889, 0.01715899,
        0.0120813 , 0.0060223 , 0.03220702, 0.00401082]))

### Formula ref

![Challenge2](https://raw.githubusercontent.com/ratikaewkam/ACC/refs/heads/main/Doc/Challenge2_doc.jpg?token=GHSAT0AAAAAACWMAKGT732ENLBU6AMFZ6TQZZGKTNA)

### Distance modulus 

In [6]:
distance = np.power(10, ((m-M+5)/5))
distance

array([1.50e+07, 9.70e+07, 3.20e+07, 1.45e+08, 5.00e+07, 1.22e+08,
       5.80e+07, 9.10e+07, 1.20e+08, 9.30e+07, 1.58e+08, 6.40e+07,
       1.45e+08, 6.10e+07, 1.03e+08, 4.60e+07, 3.40e+07, 1.85e+08,
       2.00e+07])